In [15]:
# These are all the libraries we need
import pandas as pd
import json
import re
import os
import numpy as np
from pathlib import Path
from bs4 import BeautifulSoup
from html import unescape
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
# Point to the folder where your CSVs live
DATA_DIR = Path(".")  # Change this if your CSVs are in a different folder

# QB-RAG-8 settings
NUM_QUESTIONS_PER_CHUNK = 20   # Generate 8 questions per chunk (the "8" in QB-RAG-8)
CHUNK_SIZE = 300              # Each chunk is roughly 300 words
CHUNK_OVERLAP = 50            # Chunks overlap by 50 words so we don't cut mid-sentence

In [17]:
def strip_html(text):
    """
    Your clinical data has HTML tags embedded in it like <br/>, <strong>, <p>, etc.
    This function turns that into clean readable text.
    
    Example:
      Input:  "Type of contact: Visit<br/><strong>Created by: </strong>mary"
      Output: "Type of contact: Visit Created by: mary"
    """
    if pd.isna(text):           # Handle missing/NaN values
        return ""
    text = unescape(str(text))  # Convert &amp; → &, &#8217; → ', etc.
    soup = BeautifulSoup(text, "html.parser")
    clean = soup.get_text(separator=" ", strip=True)  # Remove all HTML tags, keep text
    return re.sub(r'\s+', ' ', clean).strip()         # Collapse extra whitespace


# Quick test to see it works
raw = "Type of contact: Visit<br/><strong>Created by: </strong>mary.dumalag"
print(f"Before: {raw}")
print(f"After:  {strip_html(raw)}")

Before: Type of contact: Visit<br/><strong>Created by: </strong>mary.dumalag
After:  Type of contact: Visit Created by: mary.dumalag


In [19]:
def load_and_clean_all(data_dir):
    """
    Load all 5 CSVs, parse dates, and clean HTML from text fields.
    Returns a dictionary where each key is a table name and each value is a DataFrame.
    """
    # 1. Cycles
    cycles = pd.read_csv(data_dir / "Cycles.csv", encoding="utf-8-sig")
    cycles["Date"] = pd.to_datetime(cycles["Date"], dayfirst=True, errors="coerce")
    
    # 2. Reproduction Episodes — has HTML in several columns
    repro = pd.read_csv(data_dir / "Reproduction Episodes.csv", encoding="utf-8-sig")
    repro["Date"] = pd.to_datetime(repro["Date"], dayfirst=True, errors="coerce")
    for col in ["Therapeutic Plan", "Cause of the episode", "Obstetrics observations"]:
        if col in repro.columns:
            repro[col] = repro[col].apply(strip_html)
    
    # 3. Follow-up Notes — has HTML in the main text column
    notes = pd.read_csv(data_dir / "Follow up notes.csv", encoding="utf-8-sig")
    notes["Date"] = pd.to_datetime(notes["Date"], dayfirst=True, errors="coerce")
    notes["Follow-up_clean"] = notes["Follow-up"].apply(strip_html)
    notes = notes.sort_values("Date")
    
    # 4. Spermiogram
    sperm = pd.read_csv(data_dir / "Spermiogram.csv", encoding="utf-8-sig")
    sperm["Date"] = pd.to_datetime(sperm["Date"], dayfirst=True, errors="coerce")
    sperm["Remarks_clean"] = sperm["Remarks"].apply(strip_html)
    
    # 5. History Visits (BMI tracking)
    visits = pd.read_csv(data_dir / "History visits.csv", encoding="utf-8-sig")
    visits["Date"] = pd.to_datetime(visits["Date"], dayfirst=True, errors="coerce")
    visits = visits.sort_values("Date")
    
    return {
        "cycles": cycles,
        "reproduction_episodes": repro,
        "follow_up_notes": notes,
        "spermiogram": sperm,
        "history_visits": visits,
    }


# Run it
data = load_and_clean_all(DATA_DIR)

# Check what we got
for name, df in data.items():
    print(f"{name}: {df.shape[0]} rows, {df.shape[1]} columns")

cycles: 17 rows, 22 columns
reproduction_episodes: 1 rows, 34 columns
follow_up_notes: 557 rows, 6 columns
spermiogram: 1 rows, 16 columns
history_visits: 56 rows, 7 columns


In [29]:
"""
Patient Data Pipeline — Dual Path Setup
========================================
Path 1: Structured CSVs → SQLite (for SQL queries)
Path 2: Clinical notes  → Text chunks (for semantic search)

Run this in your Jupyter notebook after loading your CSVs into the `data` dict.
"""

import pandas as pd
import sqlite3
import re
import os


# ──────────────────────────────────────────────────────────
# PATH 1: STRUCTURED DATA → SQLite
# ──────────────────────────────────────────────────────────

def create_sqlite_database(data, db_path="clinic.db"):
    """
    Load structured CSV dataframes into a SQLite database.
    Each dataframe becomes a SQL table that can be queried directly.
    
    Args:
        data: dict of dataframes (your existing `data` variable)
        db_path: where to save the database file
    
    Returns:
        sqlite3 connection object
    """
    # Remove old database if it exists so we start fresh
    if os.path.exists(db_path):
        os.remove(db_path)
    
    conn = sqlite3.connect(db_path)
    
    # ── Define which tables are STRUCTURED (go into SQL) ──
    # These are the tables where doctors will ask numerical/filtering questions
    structured_tables = {
        "cycles": {
            "df_key": "cycles",
            "table_name": "cycles",
        },
        "reproduction_episodes": {
            "df_key": "reproduction_episodes",
            "table_name": "reproduction_episodes",
        },
        "spermiogram": {
            "df_key": "spermiogram",
            "table_name": "spermiogram",
        },
        "history_visits": {
            "df_key": "history_visits",
            "table_name": "history_visits",
        },
    }
    
    tables_created = []
    
    for key, config in structured_tables.items():
        df = data[config["df_key"]].copy()
        table_name = config["table_name"]
        
        # Clean column names: remove special chars that break SQL
        # e.g., "#COC" → "COC", "Specific Outcome " → "Specific_Outcome"
        df.columns = [
            re.sub(r'[^a-zA-Z0-9_]', '_', col.strip()).strip('_')
            for col in df.columns
        ]
        
        # Remove fully empty columns
        df = df.dropna(axis=1, how='all')
        
        # Load into SQLite
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        tables_created.append(table_name)
        
        print(f"  ✓ {table_name}: {len(df)} rows, {len(df.columns)} columns")
    
    print(f"\nDatabase saved to: {db_path}")
    print(f"Tables created: {tables_created}")
    
    return conn


def get_schema_description():
    """
    Return human-readable column descriptions for each table.
    
    This is CRITICAL for the Text-to-SQL engine — without it, the LLM
    won't know that 'COC' means oocytes or that 'MII' means mature eggs.
    The LLM reads these descriptions to write correct SQL queries.
    
    Update this as Raquel sends you new data with additional columns.
    """
    return {
        "cycles": {
            "table_description": "IVF treatment cycles with outcomes and embryology results",
            "columns": {
                "Id": "Unique cycle identifier",
                "Process": "Type of procedure (e.g., I.V.F Stimulation, Endometrial Prep.)",
                "Protocol": "Stimulation protocol used (e.g., Antagonist fixed D5, Luteal phase stimulation)",
                "Date": "Date the cycle started (DD/MM/YYYY)",
                "MD": "Treating physician initials",
                "Technique": "Lab technique (e.g., ICSI, VITRIFICATION)",
                "Ooc__Source": "Source of oocytes (Fresh Own, Combined, etc.)",
                "Semen_source": "Source of sperm (Fresh ejaculation, Frozen ejaculation, etc.)",
                "Outcome": "Cycle result (Pregnancy: Yes, No transfer: Embryo vitrification, etc.)",
                "Specific_Outcome": "Detailed outcome information",
                "Live_birth": "Whether a live birth resulted (e.g., '1, NVD' = 1 baby, normal vaginal delivery)",
                "PGT": "Pre-implantation genetic testing type (PGT-A = aneuploidy screening)",
                "Ref__No_": "Reference number for the cycle",
                "_ET": "Number of embryos transferred",
                "_COC": "Number of cumulus-oocyte complexes (oocytes) retrieved",
                "MII": "Number of metaphase II (mature) oocytes",
                "_2PN": "Number of normally fertilized oocytes (2 pronuclei)",
                "Frozen_oocytes": "Number of oocytes frozen (vitrified)",
                "Frozen_embryo": "Number of embryos frozen",
                "Transferable_Emb_": "Number of embryos suitable for transfer",
                "Age": "Patient age at time of cycle",
                "BMI": "Patient BMI at time of cycle",
            }
        },
        "reproduction_episodes": {
            "table_description": "Patient reproductive history, demographics, and episode information",
            "columns": {
                "Id": "Episode identifier",
                "Date": "Episode start date",
                "Working_Centre": "Clinic name",
                "Patient_ID": "Anonymized patient identifier",
                "Gestations": "Number of previous pregnancies",
                "Para": "Number of births",
                "Misscarriage": "Number of miscarriages",
                "E_P": "Number of ectopic pregnancies",
                "C_Sections": "Number of cesarean sections",
                "Preterm_Birth": "Number of preterm births",
                "Insurance_company": "Patient insurance provider",
                "Partner_ID": "Anonymized partner identifier",
                "Reason_Consultation": "Why the patient consulted (e.g., Primary infertility)",
                "Therapeutic_Plan": "Planned treatment approach",
                "Years_of_marriage": "Duration of marriage in years",
                "Nationality": "Patient nationality",
                "Gender": "Patient gender",
                "Marital_status": "Current marital status",
                "Cryopreserved_available": "Number of cryopreserved samples available",
                "Total_cryopreserved": "Total cryopreserved samples",
                "Cycle_length": "Menstrual cycle length in days",
                "Previous_treatments": "Number of previous fertility treatments",
                "Cause_of_the_episode": "Clinical cause/diagnosis for this episode",
            }
        },
        "spermiogram": {
            "table_description": "Semen analysis results for the male partner",
            "columns": {
                "Id": "Spermiogram test identifier",
                "Request_date": "Date the test was requested",
                "Date": "Date the test was performed",
                "External": "Whether the test was done externally",
                "Episode": "Related episode reference",
                "Patient": "Patient (male partner) identifier",
                "Volume____1_4_ml_": "Semen volume in ml (normal ≥ 1.4 ml)",
                "Count____16_M_ml_": "Sperm count in millions per ml (normal ≥ 16 M/ml)",
                "Progressive____30__": "Progressive motility percentage (normal ≥ 30%)",
                "Normal_Forms____4__": "Normal morphology percentage (normal ≥ 4%)",
                "Sperm_Motility_Recovery": "Total motile sperm after processing",
                "Sperm_Motility_Recovery_ml": "Motile sperm per ml after processing",
                "Diagnosis": "Spermiogram diagnosis (e.g., Normozoospermia, Oligozoospermia)",
                "Partner": "Female partner identifier",
                "Remarks": "Additional notes about the sample",
            }
        },
        "history_visits": {
            "table_description": "Patient visit history with anthropometric measurements over time",
            "columns": {
                "Code": "Visit identifier",
                "Patient_ID": "Anonymized patient identifier",
                "Date": "Visit date (DD/MM/YYYY)",
                "Age": "Patient age at visit",
                "Height": "Patient height in meters",
                "Weight": "Patient weight in kg",
                "BMI": "Body Mass Index (kg/m²)",
            }
        },
    }


def verify_database(conn):
    """
    Quick sanity check — print each table's row count and a sample query.
    Run this after creating the database to make sure everything loaded correctly.
    """
    cursor = conn.cursor()
    
    # List all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = [row[0] for row in cursor.fetchall()]
    
    print("Database verification:")
    print("=" * 50)
    
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        
        cursor.execute(f"PRAGMA table_info({table})")
        columns = [row[1] for row in cursor.fetchall()]
        
        print(f"\n  {table}: {count} rows, {len(columns)} columns")
        print(f"  Columns: {', '.join(columns[:8])}{'...' if len(columns) > 8 else ''}")
    
    # Run a sample query to prove it works
    print("\n" + "=" * 50)
    print("Sample queries:")
    print("-" * 50)
    
    try:
        cursor.execute("SELECT Date, Process, Technique, Outcome FROM cycles LIMIT 3")
        rows = cursor.fetchall()
        print("\n  SELECT Date, Process, Technique, Outcome FROM cycles LIMIT 3:")
        for row in rows:
            print(f"    {row}")
    except Exception as e:
        print(f"  Query failed: {e}")
    
    try:
        cursor.execute("SELECT Date, Age, BMI FROM history_visits ORDER BY Date LIMIT 5")
        rows = cursor.fetchall()
        print("\n  SELECT Date, Age, BMI FROM history_visits ORDER BY Date LIMIT 5:")
        for row in rows:
            print(f"    {row}")
    except Exception as e:
        print(f"  Query failed: {e}")
    
    try:
        cursor.execute("SELECT COUNT(*) FROM cycles WHERE Outcome LIKE '%Pregnancy%'")
        count = cursor.fetchone()[0]
        print(f"\n  Cycles resulting in pregnancy: {count}")
    except Exception as e:
        print(f"  Query failed: {e}")


# ──────────────────────────────────────────────────────────
# PATH 2: CLINICAL NOTES → TEXT CHUNKS (for semantic search)
# ──────────────────────────────────────────────────────────

def clean_html(text):
    """Strip HTML tags and clean up whitespace from clinical notes."""
    if pd.isna(text) or str(text).strip() == "":
        return ""
    text = str(text)
    text = re.sub(r'<[^>]+>', ' ', text)           # Remove HTML tags
    text = re.sub(r'&[a-zA-Z]+;', ' ', text)       # Remove HTML entities like &amp;
    text = re.sub(r'&#\d+;', ' ', text)             # Remove numeric HTML entities like &#160;
    text = re.sub(r'\s+', ' ', text).strip()        # Collapse whitespace
    return text


def build_clinical_note_chunks(patient_id, data, max_chunk_tokens=300):
    """
    Extract ONLY the unstructured clinical text and split into chunks
    for vector search. This does NOT touch the structured data — that
    lives in SQLite now.
    
    Args:
        patient_id: which patient to extract notes for
        data: dict of dataframes
        max_chunk_tokens: approximate max tokens per chunk (1 token ≈ 0.75 words)
    
    Returns:
        List of dicts, each with 'text', 'metadata' (date, type, source)
    """
    chunks = []
    max_words = int(max_chunk_tokens * 0.75)  # Rough token-to-word conversion
    
    # ── Source 1: Follow-up Notes ──
    # These are the richest source of unstructured clinical text
    notes_df = data["follow_up_notes"]
    if "Patient ID" in notes_df.columns:
        notes_df = notes_df[notes_df["Patient ID"] == patient_id]
    
    for _, row in notes_df.iterrows():
        # Use cleaned version if available, otherwise clean on the fly
        if "Follow-up_clean" in notes_df.columns:
            text = str(row.get("Follow-up_clean", ""))
        else:
            text = clean_html(row.get("Follow-up", ""))
        
        if not text or len(text.strip()) < 20:
            continue
        
        metadata = {
            "source": "follow_up_notes",
            "date": str(row.get("Date", "")),
            "type": str(row.get("Type", "")),
            "note_id": str(row.get("Id", "")),
            "patient_id": str(patient_id),
        }
        
        # Split long notes into sub-chunks with overlap
        words = text.split()
        if len(words) <= max_words:
            chunks.append({"text": text, "metadata": metadata})
        else:
            # Sliding window with ~50 token (37 word) overlap
            overlap_words = 37
            start = 0
            chunk_idx = 0
            while start < len(words):
                end = start + max_words
                chunk_text = " ".join(words[start:end])
                chunk_meta = {**metadata, "chunk_index": chunk_idx}
                chunks.append({"text": chunk_text, "metadata": chunk_meta})
                start = end - overlap_words
                chunk_idx += 1
    
    # ── Source 2: Free-text fields from Reproduction Episodes ──
    # Cause of the episode and Therapeutic Plan often contain narrative text
    episodes_df = data["reproduction_episodes"]
    if "Patient ID" in episodes_df.columns:
        episodes_df = episodes_df[episodes_df["Patient ID"] == patient_id]
    
    freetext_columns = [
        ("Cause of the episode", "episode_cause"),
        ("Therapeutic Plan", "episode_therapeutic_plan"),
        ("Obstetrics observations", "episode_obstetrics"),
    ]
    
    for col_name, source_label in freetext_columns:
        if col_name not in episodes_df.columns:
            continue
        for _, row in episodes_df.iterrows():
            text = clean_html(row.get(col_name, ""))
            if not text or len(text.strip()) < 20:
                continue
            
            metadata = {
                "source": source_label,
                "date": str(row.get("Date", "")),
                "patient_id": str(patient_id),
            }
            
            words = text.split()
            if len(words) <= max_words:
                chunks.append({"text": text, "metadata": metadata})
            else:
                overlap_words = 37
                start = 0
                chunk_idx = 0
                while start < len(words):
                    end = start + max_words
                    chunk_text = " ".join(words[start:end])
                    chunk_meta = {**metadata, "chunk_index": chunk_idx}
                    chunks.append({"text": chunk_text, "metadata": chunk_meta})
                    start = end - overlap_words
                    chunk_idx += 1
    
    # ── Source 3: Spermiogram Remarks ──
    sperm_df = data["spermiogram"]
    for _, row in sperm_df.iterrows():
        text = clean_html(row.get("Remarks", ""))
        if not text or len(text.strip()) < 20:
            continue
        chunks.append({
            "text": text,
            "metadata": {
                "source": "spermiogram_remarks",
                "date": str(row.get("Date", "")),
                "patient_id": str(patient_id),
            }
        })
    
    return chunks


def print_chunk_summary(chunks):
    """Print a summary of the generated chunks."""
    print(f"Total chunks: {len(chunks)}")
    print()
    
    # Count by source
    sources = {}
    for chunk in chunks:
        src = chunk["metadata"]["source"]
        sources[src] = sources.get(src, 0) + 1
    
    print("Chunks by source:")
    for src, count in sorted(sources.items()):
        print(f"  {src}: {count}")
    
    # Show average chunk size
    avg_words = sum(len(c["text"].split()) for c in chunks) / len(chunks) if chunks else 0
    print(f"\nAverage chunk size: {avg_words:.0f} words (~{avg_words / 0.75:.0f} tokens)")
    
    # Preview first chunk
    if chunks:
        print(f"\nSample chunk (first 200 chars):")
        print(f"  Source: {chunks[0]['metadata']['source']}")
        print(f"  Date: {chunks[0]['metadata']['date']}")
        print(f"  Text: {chunks[0]['text'][:200]}...")


# ──────────────────────────────────────────────────────────
# MAIN: Run both paths
# ──────────────────────────────────────────────────────────

def build_dual_path_pipeline(data, patient_id=1, db_path="clinic.db"):
    """
    Run the complete data ingestion pipeline:
      - Path 1: Structured tables → SQLite
      - Path 2: Clinical notes → text chunks (ready for embedding)
    
    Args:
        data: dict of dataframes (your existing loaded CSVs)
        patient_id: which patient to process
        db_path: where to save the SQLite database
    
    Returns:
        conn: SQLite connection (for structured queries)
        chunks: list of text chunks with metadata (for vector search)
        schema: column descriptions (for the LLM to understand the tables)
    """
    print("=" * 60)
    print("  DUAL-PATH DATA PIPELINE")
    print("=" * 60)
    
    # ── Path 1: Structured → SQLite ──
    print("\n📊 PATH 1: Loading structured data into SQLite...")
    print("-" * 40)
    conn = create_sqlite_database(data, db_path)
    
    print("\n🔍 Verifying database...")
    print("-" * 40)
    verify_database(conn)
    
    # ── Path 2: Notes → Chunks ──
    print("\n\n📝 PATH 2: Building clinical note chunks...")
    print("-" * 40)
    chunks = build_clinical_note_chunks(patient_id, data)
    print_chunk_summary(chunks)
    
    # ── Schema descriptions ──
    schema = get_schema_description()
    
    # ── Summary ──
    print("\n\n" + "=" * 60)
    print("  PIPELINE COMPLETE")
    print("=" * 60)
    print(f"  SQLite database : {db_path}")
    print(f"  Structured tables: {len(schema)} tables ready for SQL queries")
    print(f"  Text chunks      : {len(chunks)} chunks ready for embedding")
    print(f"  Schema descriptions: defined for all {len(schema)} tables")
    print("=" * 60)
    
    return conn, chunks, schema


# ──────────────────────────────────────────────────────────
# USAGE (paste this into your notebook cell):
# ──────────────────────────────────────────────────────────
#
conn, chunks, schema = build_dual_path_pipeline(data, patient_id=1)
#
#   # Now you can query SQL directly:
pd.read_sql("SELECT Date, Outcome FROM cycles", conn)
#
#   # And chunks are ready to embed into ChromaDB:
for chunk in chunks[:3]:
    print(chunk["text"][:100])
    print(chunk["metadata"])
    print()

  DUAL-PATH DATA PIPELINE

📊 PATH 1: Loading structured data into SQLite...
----------------------------------------
  ✓ cycles: 17 rows, 21 columns
  ✓ reproduction_episodes: 1 rows, 32 columns
  ✓ spermiogram: 1 rows, 16 columns
  ✓ history_visits: 56 rows, 7 columns

Database saved to: clinic.db
Tables created: ['cycles', 'reproduction_episodes', 'spermiogram', 'history_visits']

🔍 Verifying database...
----------------------------------------
Database verification:

  cycles: 17 rows, 21 columns
  Columns: Id, Process, Protocol, Date, MD, Technique, Ooc__Source, Semen_source...

  reproduction_episodes: 1 rows, 32 columns
  Columns: Id, Date, Working_Centre, Patient_ID, Date_of_Birth, Gestations, Para, Misscarriage...

  spermiogram: 1 rows, 16 columns
  Columns: Id, Request_date, Date, External, Episode, Patient, Volume___1_4_ml, Count___16_M_ml...

  history_visits: 56 rows, 7 columns
  Columns: Code, Patient_ID, Date, Age, Height, Weight, BMI

Sample queries:
-------------------

In [21]:
def chunk_document(text, chunk_size=CHUNK_SIZE, overlap=CHUNK_OVERLAP):
    """
    Split the patient document into overlapping pieces.
    
    Why? The full document is ~52,000 words — way too big to search 
    efficiently or fit in an LLM context. So we break it into ~300-word 
    pieces. The 50-word overlap ensures we don't lose information at 
    the boundaries between chunks.
    
    This is equivalent to the "content cards" in the QB-RAG paper.
    Each chunk will later get 8 questions generated for it.
    """
    words = text.split()
    chunks = []
    start = 0
    chunk_id = 0
    
    while start < len(words):
        end = start + chunk_size
        chunk_text = " ".join(words[start:end])
        
        chunks.append({
            "chunk_id": chunk_id,
            "text": chunk_text,
        })
        
        chunk_id += 1
        start += chunk_size - overlap  # Step forward, but keep overlap
    
    return chunks


# Run it
chunks = chunk_document(patient_doc)
print(f"Total chunks: {len(chunks)}")
print(f"Words per chunk: ~{CHUNK_SIZE}")
print(f"Overlap: {CHUNK_OVERLAP} words")

# Look at the first 2 chunks
print(f"\n--- Chunk 0 ---")
print(chunks[0]["text"][:300])
print(f"\n--- Chunk 1 ---")
print(chunks[1]["text"][:300])

Total chunks: 244
Words per chunk: ~300
Overlap: 50 words

--- Chunk 0 ---
=== PATIENT ANAMNESIS === Id: 1070 | Date: 2019-10-28 00:00:00 | Working Centre: ART Fertility Clinics | Patient ID: 1 | *Date of Birth: 28126 | Gestations: 2 | Para: 3 | Misscarriage: 0 | E.P: 0 | C Sections: 1 | Preterm Birth: 1 | Insurance company: DAMAN THIQA | Partner ID: 2 | *Date of Birth Par

--- Chunk 1 ---
CYCLES === Id: 7571 | Process: I.V.F Stimulation | Protocol: Antagonist fixed D5 | Date: 2020-01-15 00:00:00 | MD: HF | Technique: ICSI | Semen source: Frozen ejaculation | Outcome: No transfer: Embryo vitrification | PGT: PGT-A (Screening of aneuploidies) | Ref. No.: C4700 | #ET: 0 | #COC: 19.0 | M


In [25]:
# This is the prompt we send to an LLM for each chunk.
# Adapted from QB-RAG paper Appendix H.1, but for fertility clinic data
# instead of diabetes content cards.

QUESTION_GENERATION_PROMPT = """You are a fertility clinic physician. Your task is to generate {num_questions} questions that a clinician might ask, which can be answered using ONLY the provided clinical text.

INSTRUCTIONS:
- Generate exactly {num_questions} questions
- Questions should be diverse: cover different facts in the text
- Questions should be answerable from this text alone
- Questions should be in the style a doctor or nurse would ask about a patient
- Include questions about: diagnoses, lab values, treatment plans, medications, 
  outcomes, dates, measurements, and patient history where relevant
- Do NOT copy the text verbatim into the questions

EXAMPLES of good clinical questions:
- "What was the patient's BMI at their most recent visit?"
- "What was the sperm diagnosis from the latest spermiogram?"
- "Did the patient have any successful embryo transfers?"
- "What medications were prescribed during the last consultation?"

Text:
{chunk_text}

Generate exactly {num_questions} questions as a JSON list:
{{"questions": ["question1", "question2", ...]}}
"""

print("Prompt template loaded.")
print(f"Each chunk will get {NUM_QUESTIONS_PER_CHUNK} questions generated.")

Prompt template loaded.
Each chunk will get 20 questions generated.


  Error on chunk 0: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}, 'request_id': 'req_011CYNr8q9WH1MkmdaFnHyBL'}
Generated 0 questions for chunk 0:

